In [20]:
# i want to finetune a model to do credit card fraud detection. A sample input is given below.

# the huggingface dataset for legitimate transactions is called "yunfan-y/fraud-detection-legitimate"

# the huggingface dataset for fraudulent transactions is called "yunfan-y/fraud-detection-fraud"

# all datasets have columns "conversation" and "response" 
# the response is either "LEGITIMATE" or "FRAUD"

# here is a sample data: 

# conversation: Transaction Details: - Date/Time: 2019-05-26 05:20:36 - Merchant: fraud_Romaguera, Cruickshank and Greenholt - Amount: $104.9 - Category: shopping_net - Gender: M - State: OR

# response: LEGITIMATE

In [21]:
# Import libraries
import datasets
from datasets import load_dataset, concatenate_datasets
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import torch
import wandb

# Import PEFT libraries for LoRA
from peft import get_peft_model, LoraConfig, TaskType

wandb.init(project="fraud_detection")

In [22]:
# Load legitimate transactions dataset
legitimate_dataset = load_dataset("yunfan-y/fraud-detection-legitimate")

# Load fraudulent transactions dataset
fraudulent_dataset = load_dataset("yunfan-y/fraud-detection-fraud")

In [23]:
# Assign label 0 to legitimate transactions
legitimate_dataset = legitimate_dataset.map(lambda x: {'label': 0})

# Assign label 1 to fraudulent transactions
fraudulent_dataset = fraudulent_dataset.map(lambda x: {'label': 1})


# Combine the datasets
train_dataset = concatenate_datasets([legitimate_dataset['train'], fraudulent_dataset['train']])
train_dataset = train_dataset.shuffle(seed=42)
eval_dataset = concatenate_datasets([legitimate_dataset['validation'], fraudulent_dataset['validation']])
eval_dataset = eval_dataset.shuffle(seed=42)
test_dataset = concatenate_datasets([legitimate_dataset['test'], fraudulent_dataset['test']])
test_dataset = test_dataset.shuffle(seed=42)


In [24]:
# Initialize the tokenizer
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

# Tokenization function
def tokenize_function(example):
    return tokenizer(example['conversation'], padding='max_length', truncation=True)

# Apply the tokenizer to the datasets
tokenized_train = train_dataset.map(tokenize_function, batched=True)
tokenized_eval = eval_dataset.map(tokenize_function, batched=True)

In [25]:
# Load a pre-trained model for sequence classification
base_model = AutoModelForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

# Configure LoRA
lora_config = LoraConfig(
    task_type=TaskType.SEQ_CLS,  # For sequence classification
    inference_mode=False,
    r=8,
    lora_alpha=32,
    lora_dropout=0.1
)

# Wrap the base model with LoRA
model = get_peft_model(base_model, lora_config)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [26]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = torch.argmax(torch.tensor(logits), dim=-1)
    acc = accuracy_score(labels, predictions)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='binary')
    return {
        'accuracy': acc,
        'precision': precision,
        'recall': recall,
        'f1': f1,
    }

In [27]:
training_args = TrainingArguments(
    output_dir='./results',            # Output directory
    evaluation_strategy='steps',       # Evaluate every N steps
    save_strategy='steps',             # Save the model every N steps
    eval_steps=2500,                    # Evaluation interval
    save_steps=5000,                    # Save interval
    num_train_epochs=3,                # Number of training epochs
    per_device_train_batch_size=32,    # Batch size for training
    per_device_eval_batch_size=32,     # Batch size for evaluation
    logging_dir='./logs',              # Directory for logs
    logging_steps=10,
    load_best_model_at_end=True,       # Load the best model when finished training
    report_to="wandb",                 
    
)

/home/yyfsss/miniconda3/envs/mini/lib/python3.12/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [28]:
# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_eval,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,  
)

# Train the model
trainer.train()

/tmp/ipykernel_22282/491366245.py:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
2500,0.069200,0.116088,0.971258,0.872054,0.689747,0.770260
5000,0.068800,0.087328,0.975165,0.870031,0.757656,0.809964
7500,0.041700,0.076596,0.977211,0.830287,0.846871,0.838497


Could not locate the best model at ./results/checkpoint-7500/pytorch_model.bin, if you are running a distributed training on multiple nodes, you should activate `--save_on_each_node`.


TrainOutput(global_step=8064, training_loss=0.11312716998278148, metrics={'train_runtime': 4445.892, 'train_samples_per_second': 58.034, 'train_steps_per_second': 1.814, 'total_flos': 6.812077969907712e+16, 'train_loss': 0.11312716998278148, 'epoch': 3.0})

In [29]:
# Evaluate the model
evaluation_results = trainer.evaluate()

# Print evaluation results
print(evaluation_results)

{'eval_loss': 0.0748044028878212, 'eval_accuracy': 0.9771184075899916, 'eval_precision': 0.8144458281444583, 'eval_recall': 0.8708388814913449, 'eval_f1': 0.8416988416988417, 'eval_runtime': 71.7187, 'eval_samples_per_second': 149.905, 'eval_steps_per_second': 4.685, 'epoch': 3.0}


In [31]:


# upload model to huggingface
model.push_to_hub("yunfan-y/fraud-detection-model-lora-origin")


adapter_model.safetensors: 100%|██████████| 1.19M/1.19M [00:00<00:00, 2.18MB/s]


CommitInfo(commit_url='https://huggingface.co/yunfan-y/fraud-detection-model-lora-origin/commit/3541da59382bb4ece2dcee2f3f857644f66d462a', commit_message='Upload model', commit_description='', oid='3541da59382bb4ece2dcee2f3f857644f66d462a', pr_url=None, repo_url=RepoUrl('https://huggingface.co/yunfan-y/fraud-detection-model-lora-origin', endpoint='https://huggingface.co', repo_type='model', repo_id='yunfan-y/fraud-detection-model-lora-origin'), pr_revision=None, pr_num=None)